# 뉴스 크롤링 후 뉴스 요약

## gogamza/kobart-summarization 모델 사용

In [2]:
# =========================================
# 0. 필수 라이브러리 설치
# =========================================
!pip install -q transformers datasets sentencepiece accelerate
!pip install -q requests beautifulsoup4


In [3]:

import torch
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# GPU 사용 여부 확인
device = 0 if torch.cuda.is_available() else -1
print("CUDA available:", torch.cuda.is_available())

# =========================================
# 1. 네이버 뉴스 본문 크롤링 함수
# =========================================
def get_naver_news(url: str) -> str:
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    res.encoding = "utf-8"

    soup = BeautifulSoup(res.text, "html.parser")

    # 네이버 뉴스 본문 추출 (주의 >> 기사 링크 마다 div id가 조금 다름)
    article = soup.select_one("#dic_area")  # 최신 네이버 뉴스 본문
    if article is None:
        article = soup.select_one("div.article_body")  # 구형 구조 대응

    if article:
        text = article.get_text(separator="\n")
        return text.strip()
    else:
        return " 기사를 찾을 수 없습니다."

# =========================================
# 2. KoBART 요약 모델 로드
# =========================================
model_name = "gogamza/kobart-summarization"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=device
)

# =========================================
# 3. 뉴스 요약 함수
# =========================================
def summarize_news(article: str, max_len: int = 128) -> str:
    if not article or article.startswith("❌"):
        return article

    summary = summarizer(
        article,
        max_length=max_len,
        min_length=40,
        do_sample=False
    )
    return summary[0]["summary_text"]

# =========================================
# 4. 테스트 (네이버 뉴스 URL 입력)
# =========================================
# 예시: 네이버 뉴스 정치/경제 기사 하나 붙여보기
url = "https://n.news.naver.com/mnews/article/421/0007556575"  # 원하는 뉴스 링크 입력
article_text = get_naver_news(url)

print(" 원문 기사:\n", article_text[:500], "...\n")
print(" 요약 결과:\n", summarize_news(article_text))

# =========================================
# 5. 대화형 뉴스 요약 챗봇
# =========================================
while True:
    news_url = input("\n네이버 뉴스 URL 입력 (종료하려면 quit): ")
    if news_url.lower() in ["quit", "exit"]:
        break

    text = get_naver_news(news_url)
    print("\n[원문 일부]\n", text[:300], "...\n")
    print(" 요약 결과:", summarize_news(text))


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


CUDA available: False


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cpu
Your max_length is set to 128, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


 원문 기사:
 (서울=뉴스1) 사진공동취재단 = 이창용 한국은행 총재가 23일 서울 중구 한국은행에서 열린 금융통화위원회를 주재하고 있다. 2024.5.23/뉴스1 ...



Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 요약 결과:
 이창용 한국은행 총재가 23일 서울 중구 한국은행 총재가 23일 서울 중구 한국은행에서 열린 금융통화위원회를 주재하고 있다. 2024.5.23/2024.5.23   2024.5.23/뉴스1

네이버 뉴스 URL 입력 (종료하려면 quit): https://n.news.naver.com/mnews/article/138/0002202976

[원문 일부]
 AI 역량 인식 현황은 아태 평균 밑돌아








'AI 구현을 위한 전략적 계획 수립' 보고서 이미지 [ⓒ 델 테크놀로지스·IDC]
[디지털데일리 이나연 기자] 한국 기업들의 인공지능(AI) 도입 수준이 아시아·태평양 지역 평균을 웃도는 것으로 나타났다.
델 테크놀로지스는 글로벌 시장조사기관 IDC와 함께 아태 지역의 엔터프라이즈 AI 도입 동향을 심층 분석한 보고서를 19일 발표했다.
이 보고서에 따르면 한국 기업의 32%는 AI를 다양한 직무에 도입했거나 AI 역량을 경쟁우위의 핵심으로 인식했다. 이는 아태 지역 평균인  ...



Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 요약 결과: 글로벌 시장조사기관 IDC와 함께 아태 지역의 엔터프라이즈 AI 도입 동향을 심층 분석한 보고서를 19일 발표한 보고서에 따르면 한국 기업의 32%는 AI를 다양한 직무에 도입했거나 AI 역량을 경쟁우위의 핵심으로 인식했다. 이는 이는 아태 지역 평균인 24%를 크게 앞서는 수치언더반면 AI 역량에 대한 인식은 정반대였다.


KeyboardInterrupt: Interrupted by user